<a href="https://colab.research.google.com/github/the-Soke/FAW-AI-MODEL/blob/main/ONNX_export.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/the-Soke/FAW-AI-MODEL.git

Cloning into 'FAW-AI-MODEL'...
remote: Enumerating objects: 802, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 802 (delta 0), reused 0 (delta 0), pack-reused 799 (from 2)
Receiving objects: 100% (802/802), 122.22 MiB | 36.03 MiB/s, done.
Resolving deltas: 100% (29/29), done.
Updating files: 100% (750/750), done.


In [2]:
%cd FAW-AI-MODEL

/content/FAW-AI-MODEL


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving faw-model-pipeline.keras to faw-model-pipeline.keras


In [ ]:
# Step 1: Install dependencies
!pip install tf2onnx onnx onnxruntime


INFO: pip is looking at multiple versions of onnx to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 129.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 94.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.71.2 requires protobuf<6.0dev

In [ ]:
# Step 2: Import required libraries
import tensorflow as tf
from tensorflow import keras
import numpy as np
import onnxruntime as ort


In [ ]:
# Step 3: Load your existing Keras model
model = keras.models.load_model("FAW-AI-MODEL/faw-model-pipeline.keras")
print("✅ Original model loaded successfully!")
model.summary()


✅ Original model loaded successfully!


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 178, 178, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 87, 87, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 41, 41, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 51200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     6,553,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 19,942,091 (76.07 MB)

 Trainable params: 6,647,363 (25.36 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 13,294,728 (50.72 MB)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras

# Define a function to recursively get clean layers
def get_clean_layers_recursively(keras_model_or_layer):
    clean_layers_list = []
    if hasattr(keras_model_or_layer, 'layers'): # If it's a Keras Model or Sequential layer
        for layer in keras_model_or_layer.layers:
            is_augmentation_layer = False
            # Check for common Keras data augmentation layers by type and name
            if isinstance(layer, layers.RandomFlip) or \
               isinstance(layer, layers.RandomRotation) or \
               isinstance(layer, layers.RandomZoom) or \
               isinstance(layer, layers.RandomContrast) or \
               isinstance(layer, layers.RandomTranslation) or \
               isinstance(layer, layers.RandomHeight) or \
               isinstance(layer, layers.RandomWidth) or \
               "random" in layer.name.lower() or \
               "preprocessing" in layer.name.lower():
                print(f"Skipping data augmentation/preprocessing layer: {layer.name}")
                is_augmentation_layer = True

            if not is_augmentation_layer:
                # If it's a nested Sequential/Model layer, recurse
                if isinstance(layer, keras.Sequential):
                    nested_clean_layers = get_clean_layers_recursively(layer)
                    if nested_clean_layers:
                        # Reconstruct the nested sequential model with only its clean layers
                        clean_layers_list.append(keras.Sequential(nested_clean_layers, name=layer.name))
                elif isinstance(layer, keras.Model):
                    # For a nested Functional Model, rebuilding is complex.
                    # We append it as-is, assuming it's the base model or free of augmentation.
                    print(f"Warning: Appending nested Functional model '{layer.name}' as-is. Ensure it's augmentation-free.")
                    clean_layers_list.append(layer)
                else:
                    clean_layers_list.append(layer)
    else: # If it's a single layer (e.g., an InputLayer passed initially)
        clean_layers_list.append(keras_model_or_layer)
    return clean_layers_list

# Get the clean layers from the original model
all_clean_layers = get_clean_layers_recursively(model)

# Rebuild a clean Sequential model from the filtered layers
# The input shape for the clean model (non-augmented) is (180, 180, 3) based on previous outputs.
input_shape_for_clean_model = (180, 180, 3)

final_clean_model_layers = []

# If the first layer in `all_clean_layers` is not an InputLayer, or its shape doesn't match,
# explicitly add an InputLayer to ensure the clean model has a proper entry point.
# Corrected the isinstance check from keras.Input to keras.layers.InputLayer
# and added keras.layers.InputLayer when appending.
if not all_clean_layers or not (isinstance(all_clean_layers[0], keras.layers.InputLayer) and all_clean_layers[0].input_shape[1:] == input_shape_for_clean_model):
    print(f"Adding explicit Input layer with shape: {input_shape_for_clean_model}")
    final_clean_model_layers.append(keras.layers.InputLayer(input_shape=input_shape_for_clean_model, name="input_for_inference")) # Naming for clarity

final_clean_model_layers.extend(all_clean_layers)

# If no layers remain after filtering, raise an error
if not final_clean_model_layers:
    raise ValueError("No layers remaining after attempting to remove data augmentation layers. Check model structure.")

clean_model = keras.Sequential(final_clean_model_layers, name="faw_clean_inference_model")
clean_model.summary()

print("✅ Data augmentation layers removed successfully!")

Skipping data augmentation/preprocessing layer: random_flip
Skipping data augmentation/preprocessing layer: random_rotation
Skipping data augmentation/preprocessing layer: random_zoom
Adding explicit Input layer with shape: (180, 180, 3)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/input_layer.py:27: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Model: "faw_clean_inference_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 178, 178, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 89, 89, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 87, 87, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 43, 43, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 41, 41, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 20, 20, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 51200)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     6,553,728 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,647,363 (25.36 MB)

 Trainable params: 6,647,363 (25.36 MB)

 Non-trainable params: 0 (0.00 B)

✅ Data augmentation layers removed successfully!


In [ ]:
# Step 5: Save as TensorFlow SavedModel
clean_model.export("saved_model_tf")

print("✅ Clean SavedModel created at: saved_model_tf/")


Saved artifact at 'saved_model_tf'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 180, 180, 3), dtype=tf.float32, name='input_for_inference')
Output Type:
  TensorSpec(shape=(None, 3), dtype=tf.float32, name=None)
Captures:
  134921074116560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134921074118288: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134921074118672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134921074120208: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134921074120016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134921074120784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134921074119440: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134921074117520: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134921074120976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134921074118864: TensorSpec(shape=(), dtype=tf.resource, name=None)
✅ Clean Sav

In [ ]:
!pip install numpy==1.26.4 --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 34.5 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
yd

In [ ]:
!pip install -U tf2onnx

In [ ]:
!python3 -m tf2onnx.convert \
    --saved-model saved_model_tf \
    --opset 13 \
    --output faw_model.onnx


2025-11-06 13:32:06.981356: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762435927.010148    5155 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762435927.019626    5155 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762435927.046084    5155 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762435927.046133    5155 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1762435927.046142    5155 computation_placer.cc:177] computation placer alr

# **Testing the ONNX**

In [ ]:
# Step 7: Test the ONNX model
import onnxruntime as ort
import numpy as np

session = ort.InferenceSession("faw_model.onnx")

# Get input name and shape
input_name = session.get_inputs()[0].name
print("Input name:", input_name)

# Create dummy input (shape 1,180,180,3)
dummy = np.random.rand(1, 180, 180, 3).astype(np.float32)

# Run inference
output = session.run(None, {input_name: dummy})
print("✅ ONNX model ran successfully!")
print("Output shape:", np.array(output).shape)

Input name: input_for_inference
✅ ONNX model ran successfully!
Output shape: (1, 1, 3)


In [ ]:
from google.colab import files
files.download("faw_model.onnx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# **Pushing to github**

In [ ]:
!mv faw_model.onnx FAW-AI-MODEL/
!mv saved_model_tf FAW-AI-MODEL/
!cp -r sample_data FAW-AI-MODEL/

In [3]:
from google.colab import auth
auth.authenticate_user()


In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
!cp "/content/drive/MyDrive/Colab Notebooks/ONNX-export.ipynb" /content/FAW-AI-MODEL/

In [24]:
!rm ONNX-export.ipynb


In [21]:
!cp /content/ONNX-export.ipynb FAW-AI-MODEL

cp: cannot stat '/content/ONNX-export.ipynb': No such file or directory


In [10]:
!git config --global user.email "kesibosoke@gmail.com"
!git config --global user.name "the-Soke"

In [22]:
%cd FAW-AI-MODEL
!git add .
!git commit -m "Added ONNX export notebook"

[Errno 2] No such file or directory: 'FAW-AI-MODEL'
/content/FAW-AI-MODEL
[main f52e434] Added ONNX export notebook
 1 file changed, 1 insertion(+), 1 deletion(-)


In [23]:
!git push origin main

Enumerating objects: 7, done.
Counting objects: 100% (7/7), done.
Delta compression using up to 2 threads
Compressing objects: 100% (6/6), done.
Writing objects: 100% (6/6), 14.01 KiB | 2.33 MiB/s, done.
Total 6 (delta 3), reused 0 (delta 0), pack-reused 0
remote: Resolving deltas: 100% (3/3), completed with 1 local object.
remote: error: GH013: Repository rule violations found for refs/heads/main.
remote: 
remote: - GITHUB PUSH PROTECTION
remote:   —————————————————————————————————————————
remote:     Resolve the following violations before pushing again
remote: 
remote:     - Push cannot contain secrets
remote: 
remote:     
remote:      (?) Learn how to resolve a blocked push
remote:      https://docs.github.com/code-security/secret-scanning/working-with-secret-scanning-and-push-protection/working-with-push-protection-from-the-command-line#resolving-a-blocked-push
remote:     
remote:     
remote:       —— GitHub Personal Access Token ——————————————————————
remote:        locations: